In [1]:
# Load packages

import os
os.environ['MKL_SERVICE_FORCE_INTEL'] = '1'

# general
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

# math
from scipy.stats import pearsonr
import math

# my functions/classes
import sys
sys.path.append("../core_scripts/")
from ECMclass import ECM


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Setup

In [5]:
# Set filepaths

path_to_data = '../../data/ecm/'
path_to_angles = '../../data/angles/'
path_to_figures = '../../../figures/angles/'

In [6]:
# smoothing window
window = 1

In [7]:
# Load metadata
meta = pd.read_csv(path_to_data + 'metadata.csv')
meta.head()


,core,time,section,face,ACorDC,Y_left,Y_right,AC_edgespace,DC_edgespace
0,alhic1901,cmc1,230_4,t,DC,-5.080,206.763,15,10
1,alhic1901,cmc1,230_4,l,DC,70.404,170.096,15,10
2,alhic1901,cmc1,228_4,t,DC,-10.000,192.000,15,10
3,alhic1901,cmc1,228_4,l,AC,63.881,166.250,15,10
4,alhic1901,cmc1,228_4,l,DC,65.891,149.177,15,10


In [8]:
#Load ECM data

data = []
cores = []
sections = []
faces = []
ACorDCs = []
for index, row in tqdm(meta.iterrows(), total=len(meta), desc="Processing data"):
    
    core = row['core']
        
    section = row['section']
    face = row['face']
    ACorDC = row['ACorDC']

    data_item = ECM(core,section,face,ACorDC)
    print("Reading "+core+", section "+section+'-'+face+'-'+ACorDC)
    
    data_item.rem_ends(15)
    data_item.smooth(window)
    data.append(data_item)
    data_item.norm_all()
    
    cores.append(core)
    sections.append(section)
    faces.append(face)
    ACorDCs.append(ACorDC)


Processing data:   8%|▊         | 1/13 [00:00<00:02,  4.65it/s]

Reading alhic1901, section 230_4-t-DC


Processing data:  15%|█▌        | 2/13 [00:00<00:01,  6.75it/s]

Reading alhic1901, section 230_4-l-DC
Reading alhic1901, section 228_4-t-DC


Processing data:  31%|███       | 4/13 [00:00<00:01,  5.55it/s]

...

Processing data:  85%|████████▍ | 11/13 [00:02<00:00,  4.70it/s]

Reading alhic1901, section 228_4-o-AC


Processing data:  92%|█████████▏| 12/13 [00:02<00:00,  4.10it/s]

Reading alhic1901, section 228_4-t-AC


Processing data: 100%|██████████| 13/13 [00:03<00:00,  3.91it/s]


## Make a rotated plot of a single dataset

let's start with alhic1901 228_4

In [ ]:
# define plotting function
def plot_ECM(y_vec,ycor,d,meas,button,axs,rescale,angle,face,res):

    y_vec = data.y_vec
    ycor = data.y
    d = data.depth
    meas = data.meas
    button = data.button

    ACpltmin = np.percentile(meas,5)
    ACpltmax = np.percentile(meas,95)
    rescale = lambda k: (k-ACpltmin) /  (ACpltmax-ACpltmin)

    
    # calculate track width (for plotting)
    width = y_vec[1] - y_vec[0]

    for y in y_vec:
        
        # Pull out data for this track
        idx = ycor==y
        tmeas = meas[idx]
        tbut = button[idx]
        td = d[idx]

        # downsample ECM to save plotting time (as needed)
        if res != 0:
            int_lo = round(min(td),2)
            int_hi = round(max(td),2)
            depth_interp = np.linspace(int_lo,int_hi,int((int_hi-int_lo)/res)+1)
            meas_interp = np.interp(depth_interp,np.flip(td),np.flip(tmeas))
            but_interp = np.interp(depth_interp,np.flip(td),np.flip(tbut))
            td = depth_interp
            tmeas = meas_interp
            tbut = np.round(but_interp)

        offset = y - (max(y_vec)+min(y_vec))/2

        cor = offset/1000 * np.tan(angle*np.pi/180)
        td = td + cor
        
        for i in range(len(tmeas)-1):
            
            if tbut[i] == 0:
                axs.add_patch(Rectangle((y-(width-0.2)/2,td[i]),(width-0.2),td[i+1]-td[i],facecolor=my_cmap(rescale(tmeas[i]))))
            else:
                axs.add_patch(Rectangle((y-(width-0.2)/2,td[i]),(width-0.2),td[i+1]-td[i],facecolor=my_cmap(rescale(tmeas[i]))))
            
    return()